# RNN


#### 这里演示每个cell都有预测值
$ S^{t} = tanh(U\cdot X_t + W\cdot S^{t-1} + b_s) $  
$ O^{t} = softmax(V\cdot S^{t} + b_o) $


$ S^{t} $ : 当前时刻的隐层输出 
$ O^{t} $ : 当前时刻预测值，或者叫输出值  
$ X_t $ : 当前时刻的输入值.   
$ S^{t-1} $: 上一个时刻的隐藏输出  
$ b_s $: 隐层输出的偏置项  
$ b_o $: 预测输出的偏置项  


In [5]:
import numpy as np
import torch

# 单个cell的向前传播
# s_prev: 上一个时刻的隐藏输出
def rnn_cell_forward(x_t, s_prev, parameters):
    # 取出参数
    U = parameters['U']
    W = parameters['W']
    V = parameters['V']
    b_s = parameters['b_s']
    b_o = parameters['b_o']

    # 写公式
    S_t = np.tanh(np.dot(U,X_t) + np.dot(W, S_prev) +b_s)
    O_t = softmax(np.dot(V, S_t) + b_o)
    
    # 记录每时刻值，用于反向传播
    cache = (S_t,S_prev,X_t, parameters)
    return S_t, O_t, cache


def softmax(x):
    """
    x: 一个n维向量
    """
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x

### 测试单个cell

In [6]:
############ 测试单个cell
if __name__ == '__main__':
    # 定义输入 这里定义输入词向量的维度是3， S_t的维度指定为 (5,1)
    X_t = np.random.randn(3,1)
    S_prev = np.random.randn(5,1)
    
    # 定义参数
    U = np.random.randn(5,3)
    W = np.random.randn(5,5)

    # 定义输出维度，和中间状态维度
    # 这里表示输出维度和输入的词向量维度相同都是3，可以改成不同的值
    # 比如，如果是大语言模型，输入词向量的维度是3维的，但要预测的是在整个词表(比如3万个词)中应该输出词的概率,那就可以改成3万
    #      如果是情感分析，比如喜、怒，那就是改成2
    #      如果是10分类就改成10
    m_o = 3 #表示输出维度
    V = np.random.randn(m_o,5) 
    
    b_s = np.random.randn(5,1)
    b_o = np.random.randn(m_o,1)

    parameters = {'U':U, 'W':W, 'V':V, 'b_s':b_s, 'b_o':b_o}
    
    S_t, O_t, cache = rnn_cell_forward(X_t, S_prev, parameters)
    print('S_t = ', S_t)
    print('S_t.shape = ', S_t.shape)
    print('O_t = ', O_t)
    print('O_t.shape = ', O_t.shape)

S_t =  [[ 0.84923863]
 [-0.9847439 ]
 [-0.10344664]
 [-0.32890503]
 [ 0.7684728 ]]
S_t.shape =  (5, 1)
O_t =  [[0.00953388]
 [0.20849545]
 [0.78197068]]
O_t.shape =  (3, 1)


### 测试多个cell

In [22]:
# rnn前向传播，多个cell,也就是可以接收多个单词序列
# x: 输入值，形状是(m,1 ,T) 其中(m,1)是输入的词向量形状; T是输入的单词个数(或序列长度、时刻长度)
# s0: 初始隐藏输入
# parameters:  共享的U,W,V,b_s,b_o
def rnn_forward(x, s0, parameters):
    # 获取词向量维度、单词个数
    m,_,T = x.shape
    
    # 获取 预测值的输出维度，和隐层中间状态大小(这个是自定义的数字)
    m_o,n = parameters['V'].shape
    
    # 初始状态值
    S_prev = s0
    
    # 保留所有cell在每个时刻的隐层状态和预测输出, 用S、Y保留
    S = np.zeros((n,1, T))
    Y = np.zeros((m_o,1, T)) #注意，这里(m_o,1)是定义预测值的输出维度
    
    # 存放所有缓存的列表
    caches = []
    
    # 循环每个cell进行前向传播计算
    for t in range(T):
        # t表示第几个单词
        S_t, O_t, cache = rnn_cell_forward(x[:,:,t], S_prev, parameters)
    
        S[:,:, t] = S_t #放入第t个时刻的隐层状态
        Y[:,:, t] = O_t #让如第t个时刻的预测结果
    
        # 放入缓存到列表
        caches.append(cache)
        
    return S, Y, caches 

############ 测试多个cell
if __name__ == '__main__':
    # 定义输入 
    # 假设输入是 4个单词，每个词向量是3个维度
    # 则 = (m,1, T) -> (3,1, 4)
    X = np.random.randn(3,1, 4)
    # 中间状态n=5
    S_0 = np.random.randn(5,1)
    
    # 定义参数
    U = np.random.randn(5,3)
    W = np.random.randn(5,5)

    # 定义输出维度，和中间状态维度
    # 这里表示输出维度和输入的词向量维度相同都是3，可以改成不同的值
    # 比如，如果是大语言模型，输入词向量的维度是3维的，但要预测的是在整个词表(比如3万个词)中应该输出词的概率,那就可以改成3万
    #      如果是情感分析，比如喜、怒，那就是改成2
    #      如果是10分类就改成10
    m_o = 5 #表示输出维度
    V = np.random.randn(m_o,5) 
    
    b_s = np.random.randn(5,1)
    b_o = np.random.randn(m_o,1)

    parameters = {'U':U, 'W':W, 'V':V, 'b_s':b_s, 'b_o':b_o}
    
    S, Y, caches = rnn_forward(X, S_0, parameters)
    print('S = ', S)
    print('S.shape = ', S.shape)
    print('Y = ', Y)
    print('Y.shape = ', Y.shape)

S =  [[[-0.74129897 -0.74129897 -0.74129897 -0.74129897]]

 [[ 0.99392395  0.99392395  0.99392395  0.99392395]]

 [[ 0.44604313  0.44604313  0.44604313  0.44604313]]

 [[-0.39941614 -0.39941614 -0.39941614 -0.39941614]]

 [[-0.96116777 -0.96116777 -0.96116777 -0.96116777]]]
S.shape =  (5, 1, 4)
Y =  [[[0.02374666 0.02374666 0.02374666 0.02374666]]

 [[0.06400777 0.06400777 0.06400777 0.06400777]]

 [[0.05730302 0.05730302 0.05730302 0.05730302]]

 [[0.83136845 0.83136845 0.83136845 0.83136845]]

 [[0.0235741  0.0235741  0.0235741  0.0235741 ]]]
Y.shape =  (5, 1, 4)
